Chapter 2 Derivatives and Gradients
========

Example 2.1. Symbolic differentiation provides analytical derivatives.
----

In [5]:
import sympy as sp

In [6]:
x = sp.symbols('x')
f = x**2 + x/2 - sp.sin(x)/x
f_prime = sp.diff(f, x)

In [7]:
print("Function:")
sp.pprint(f)
print("\nDerivative:")
sp.pprint(f_prime)

Function:
 2   x   sin(x)
x  + ─ - ──────
     2     x   

Derivative:
      1   cos(x)   sin(x)
2⋅x + ─ - ────── + ──────
      2     x         2  
                     x   
